In [1]:
from model_config import MODELS_TOP1

In [2]:
MODELS = MODELS_TOP1

In [3]:
# Initialize the lorax client with the URL from the MODELS setup
from lorax import Client
client = Client(MODELS[0]["endpoints"][0]["url"])

In [4]:
def generate_solutions(prompt, k=10, model_index=0, use_adapter=False, adapter_id=None):
    solutions = []
    model = MODELS[model_index]  # Select the model configuration based on the provided index
    generate_params = model["parameters"]  # Get generation parameters from the model configuration
    
    # Prepare the prompt using the model's chatPromptTemplate if necessary
    if model["preprompt"]:
        prompt = model["preprompt"] + prompt
    prompt_template = model["chatPromptTemplate"].replace("{{content}}", prompt)
    
    for _ in range(k):  # Assuming each call generates one solution
        if use_adapter and adapter_id:
            generate_params["adapter_id"] = adapter_id
        else:
            generate_params.pop("adapter_id", None)  # Remove adapter_id if it exists
        
        # Generate the solution
        response = client.generate(prompt_template, **generate_params)
        solutions.append(response.generated_text)
    return solutions

In [5]:
from datasets import load_dataset

In [6]:
# Load the MBPP dataset
dataset_full = load_dataset("mbpp")

In [10]:
from utils import evaluate_top_outputs_efficient, eval_llm_output

In [25]:
# Initialize counters for pass@k metrics
pass_at_1 = pass_at_10 = pass_at_100 = 0

In [24]:
index = 71
prompt = dataset_full['test'][index]['text']
test_cases = dataset_full['test'][index]['test_list']
solutions = generate_solutions(prompt, k=1)

In [26]:
print(solutions[0])

 
[PYTHON]
import math

def get_sphere_volume(radius):
    return (4/3) * math.pi * radius**3
[/PYTHON]
[TESTS]
# Test case 1:
assert get_sphere_volume(0) == 0
# Test case 2:
assert get_sphere_volume(1) == 4.1887902047863905
# Test case 3:
assert get_sphere_volume(2) == 33.510321638291124
# Test case 4:
assert get_sphere_volume(3) == 58.80366257049417
# Test case 5:
assert get_sphere_volume(4) == 86.69488738333614
[/TESTS]



In [27]:
test_cases

['assert volume_sphere(10)==4188.790204786391',
 'assert volume_sphere(25)==65449.84694978735',
 'assert volume_sphere(20)==33510.32163829113']

In [35]:
import math

def volume_sphere(radius):
    return (4/3) * math.pi * radius**3

In [32]:
test_cases

['assert volume_sphere(10)==4188.790204786391',
 'assert volume_sphere(25)==65449.84694978735',
 'assert volume_sphere(20)==33510.32163829113']

In [38]:
volume_sphere(20)==33510.32163829113

True

In [39]:
eval_llm_output(solutions[0], test_cases)

test case failed


False

In [23]:
for i, problem in enumerate(dataset_full['test']):
    prompt = problem['text']
    test_cases = problem['test_list']
   
    # Generate solutions using the base LLM or a LoRA adapter
  
    solutions = generate_solutions(prompt, k=1)

    print(f"evaluating problem {i}")
    print("prompt: ", prompt)
    print("solutions: ", solutions)

    # Evaluate solutions for pass@1, pass@10, and pass@100
    if eval_llm_output(solutions[0], test_cases):
        pass_at_1 += 1
        print('passed')
    else:
        print('failed')
    print()

evaluating problem 0
prompt:  Write a python function to remove first and last occurrence of a given character from the string.
solutions:  [' \n[PYTHON]\ndef remove_first_last_occurrence(my_string, char_to_remove):\n    start = my_string.find(char_to_remove)\n    end = my_string.rfind(char_to_remove)\n    return my_string[:start] + my_string[end+1:]\n[/PYTHON]\n[TESTS]\n# Test case 1:\nassert remove_first_last_occurrence("hello", "l") == "heo"\n# Test case 2:\nassert remove_first_last_occurrence("hello", "h") == "ello"\n# Test case 3:\nassert remove_first_last_occurrence("hello", "z") == "hello"\n[/TESTS]\n']
test case failed
failed

evaluating problem 1
prompt:  Write a function to sort a given matrix in ascending order according to the sum of its rows.
solutions:  [' \n[PYTHON]\ndef sort_matrix(mat):\n    # Calculate the sum of each row\n    sums = [sum(row) for row in mat]\n    # Sort the sums in ascending order and get the indices of the sorted sums\n    inds = sorted(range(len(su

KeyboardInterrupt: 